<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Configuring-pandas" data-toc-modified-id="Configuring-pandas-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Configuring pandas</a></span></li><li><span><a href="#데이터-집계" data-toc-modified-id="데이터-집계-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>데이터 집계</a></span></li><li><span><a href="#분할-적용-조합(SAC)-패턴" data-toc-modified-id="분할-적용-조합(SAC)-패턴-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>분할-적용-조합(SAC) 패턴</a></span></li><li><span><a href="#Data-for-the-examples" data-toc-modified-id="Data-for-the-examples-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Data for the examples</a></span></li></ul></div>

# Configuring pandas

In [1]:
# import numpy and pandas
import numpy as np
import pandas as pd

# used for dates
import datetime
from datetime import datetime, date

# Set some pandas options controlling output format
pd.set_option('display.notebook_repr_html', False)
pd.set_option('display.max_columns', 8)
pd.set_option('display.max_rows', 10)
pd.set_option('display.width', 60)

# bring in matplotlib for graphics
import matplotlib.pyplot as plt
%matplotlib inline

# version check
print(pd.__version__)
print(np.__version__)
import sys
print(sys.version)

1.4.1
1.22.4
3.8.5 (default, Sep  4 2020, 02:22:02) 
[Clang 10.0.0 ]


# 데이터 집계
---
데이터 집계(data aggregation)는 정보의 유의미한 범주를 기준으로 데이터를 그룹화하는 과정이다.  
그다음엔 각 그룹에 대해 하나 이상의 **요약 통계를 리포팅**하기 위한 분석이 이루어진다. 그런 맥락에서의 요약은 일반적인 용어라고 할 수 있다. 요약(summarization)이라는 말이 문자 그대로 총합(summation, 총 sales 같은 것)도 의미하지만, 평균이나 표준편차와 같은 통계적 계산도 의미하기 때문이다.  

이번엔 pandas로 데이터 집계를 수행하는 방법을 배운다. 여기에는 강력한 분할-적용-조합(split-apply-combine) 패턴을 통한 그룹화, 그룹 단위의 변형과 분석 수행, 각 그룹의 결과를 하나의 요약 객체로 리포팅하는 내용이 포함된다. 또한 그와 같은 프레임워크 안에서 데이터 그룹화, 그룹 단위의 함수 적용, 데이터 필터링과 같은 여러 기법을 시험해본다.  

- 데이터 분석을 위한 분할-적용-조합 패턴의 개요
- 단일 칼럼의 그룹화
- 그룹화된 결과 접근
- 복수 칼럼의 그룹화
- 인덱스 레벨을 이용한 그룹화
- 인덱스 레벨을 이용한 그룹화
- 그룹화된 데이터에 집계 함수 적용
- 데이터 변형의 개요
- 데이터 변형의 실전 예제: 평균과 표준 점수(z-score)
- 데이터 그룹의 선택적 제거를 위한 필터링 
- 이산화와 구간화(binning)


# 분할-적용-조합(SAC) 패턴

split-apply-combine 패턴은 여러 데이터 분석에 있어서 자주 활용되는 데이터 처리 패턴이다. 이 패턴에는 다음과 같은 데이터 분석의 3단계가 있다.  
- 어떤 기준에 따라 데이터 셋을 작은 조각으로 분할한다.
- 각 조각에 대해 독립적으로 연산을 수행한다.
- 모든 결과를 다시 하나의 단위로 조합한다. 

예를 들어, 문자 기준의 키(a, b)로 그룹화된 값들의 평균을 구하는 과정에서 a, b로 나누어 평균을 구하여 적용하고, a와 a그룹의 평균, b와 b그룹의 평균을 조합하여 보여주는 식이다.  
먼저 인덱스 레이블에 따라 두 그룹으로 분할된다. 그 다음에 그 그룹의 평균이 계산된다. 각 그룹의 결과는 인덱스 레이블과 함께 다시 하나의 pandas 객체로 조합된다.  

pandas에서 분할 작업은 `Series`나 `DataFrame`의 `.groupby()` 메서드로 수행할 수 있다.  
이 메서드에는 하나 이상의 인덱스 레이블이나 칼럼명을 지정할 수 있으며, 이에 기초해 연관된 값들이 그룹화된다.  

데이터가 분할된 다음에는 각 그룹에 대해 다음과 같은 작업들이 가능하다.  
- 집계: 각 그룹의 아이템에 대한 평균이나 개수와 같은 요약 평균을 계산한다. 
- 변형: 그룹이나 아이템에 특정적인 계산을 수행한다.
- 필터링: 그룹 단위의 계산에 기초해 불필요한 데이터 그룹을 제거한다. 

마지막 조합 단계는 pandas에 의해 자동으로 수행되며, 적용 단계의 각 결과를 모아 하나의 통합된 결과를 만들어낸다.  

    SAC에 대한 자세한 내용은 <Journal of Statistical Software> 의 '데이터 분석을 위한 SAC 전략(The Split-Apply-Combine Strategy for Data Analysis)' ㄹ는 논문을 참고하기 바란다. R을 사용하는 예제로 SAC 패턴을 설명하고는 있지만, pandas 입장에서도 유용하다.  
    다운로드: [http://www.jstatsoft.org/v40/i01/paper]

# Data for the examples
두 기기의 센서에서 얻은 측정치로 된 데이터셋이다. 여기에는 가속도계와 방향 센서 각각의 X, Y, Z 축 측정 데이터가 포함돼 있다.  

In [2]:
# load the sensors data
sensor_data = pd.read_csv("/Users/Angela/Desktop/Personal/Learning-Pandas-Second-Edition-master/data/sensors.csv")
sensor_data[:5]

   interval sensor axis  reading
0         0  accel    Z      0.0
1         0  accel    Y      0.5
2         0  accel    X      1.0
3         1  accel    Z      0.1
4         1  accel    Y      0.4